In [1]:
import numpy as np
import scipy as sp
import pandas as pd
#import matplotlib.pyplot as plt
import duckdb
#from mpl_toolkits import mplot3d 
#from mpl_toolkits.mplot3d import Axes3D

# Dividing dataset into training and testing dataset and standarized the features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA

In [2]:
periodo = 365
USE_CUMSUM = True
grupo_etario_list=["Menores_1","De_1_a_4","De_5_a_14","De_15_a_64","De_65_y_mas"]
grupo_etario_legend=["Menores 1","De 1 a 4","De 5 a 14","De 15 a 64","De 65 y mas"]
#grupos_etarios = ["Menores_1","De_1_a_4","De_5_a_14","De_15_a_64","De_65_y_mas"]

In [3]:
# common_filter = """
# FROM datos_urgencia_consolidados.parquet
# where IdCausa in (3,4,5,6,10,11)
# and CodigoDependencia is not null
# --and GLOSATIPOESTABLECIMIENTO = 'Hospital'
# and year(strptime(fecha, '%d/%m/%Y')) in (2017,2018,2019,2022,2023)
# group by CodigoDependencia, fecha
# """ + """
# having dayofyear(strptime(fecha, '%d/%m/%Y')) <= """+ str(periodo) + """
# """

# df2 = duckdb.sql("""
# select year(strptime(fecha, '%d/%m/%Y')) as year
#     , CodigoDependencia  as CodigoRegion
#     , dayofyear(strptime(fecha, '%d/%m/%Y')) as nrodia
#     , 'Menores_1' as grupo_etario
#     , sum(Menores_1) as atenciones
# """+ common_filter + """
# UNION
# select year(strptime(fecha, '%d/%m/%Y')) as year
#     , CodigoDependencia  as CodigoRegion
#     , dayofyear(strptime(fecha, '%d/%m/%Y')) as nrodia
#     ,'De_1_a_4' as grupo_etario
#     , sum(De_1_a_4) as atenciones
# """+ common_filter + """
# UNION
# select year(strptime(fecha, '%d/%m/%Y')) as year
#     , CodigoDependencia  as CodigoRegion
#     , dayofyear(strptime(fecha, '%d/%m/%Y')) as nrodia
#     , 'De_5_a_14' as grupo_etario
#     , sum(De_5_a_14) as atenciones
# """+ common_filter + """
# UNION
# select year(strptime(fecha, '%d/%m/%Y')) as year
#     , CodigoDependencia  as CodigoRegion
#     , dayofyear(strptime(fecha, '%d/%m/%Y')) as nrodia
#     , 'De_15_a_64'  as grupo_etario
#     , sum(De_15_a_64) as atenciones
# """+ common_filter + """
# UNION
# select year(strptime(fecha, '%d/%m/%Y')) as year
#     , CodigoDependencia  as CodigoRegion
#     , dayofyear(strptime(fecha, '%d/%m/%Y')) as nrodia
#     , 'De_65_y_mas' as grupo_etario
#     , sum(De_65_y_mas) as atenciones
# """+ common_filter + """
# """).df()



In [4]:
#read data from parquet file
#number of days from the beginning of the year to consider in the analysis

query_list = []

for k, gr_etario in enumerate(grupo_etario_list):
    query_list.append("""
select year(strptime(fecha, '%d/%m/%Y')) as year
    , CodigoDependencia  as CodigoRegion
    , dayofyear(strptime(fecha, '%d/%m/%Y')) as nrodia
    , """ + "'" + gr_etario +"'"+ """ as grupo_etario
    , sum("""+ gr_etario + """) as atenciones
FROM datos_urgencia_consolidados.parquet
where IdCausa in (3,4,5,6,10,11)
and CodigoDependencia is not null
and year(strptime(fecha, '%d/%m/%Y')) in (2017,2018,2019,2022,2023)
group by CodigoDependencia, fecha
having dayofyear(strptime(fecha, '%d/%m/%Y')) <= """+ str(periodo) )
    if k +1< len(grupo_etario_list):
        query_list.append(""" UNION """)

df2 = duckdb.sql(''.join(query_list)).df()


df2 = df2.sort_values(by=['year','CodigoRegion','grupo_etario','nrodia']).reset_index(drop = True)

In [5]:
#the cumulative sum is taken on each of the classes, 
#is this case we group by "year", "CodigoRegion" and "grupo_etario"
#the result is a dataframe with the "nro_dia" in the columns, 
#in which the cumulative information of the demand, scaled from 0 to 1 is stored
groups = df2[["year","CodigoRegion","grupo_etario"]].drop_duplicates()
df_aux =df2.copy()
for (index, year, region, grupo_edad) in groups.itertuples():
    indicesToKeep = (df_aux.loc[:,'year'] == year) & (df_aux.loc[:,'CodigoRegion'] == region) & (df_aux.loc[:,'grupo_etario'] == grupo_edad)
    if sum(indicesToKeep)>0:
        if USE_CUMSUM:
            df_aux.loc[indicesToKeep, 'atenciones'] = df_aux.loc[indicesToKeep, 'atenciones'].cumsum()
            df_aux.loc[indicesToKeep, 'atenciones'] = df_aux.loc[indicesToKeep, 'atenciones'] / max(df_aux.loc[indicesToKeep, 'atenciones'])
        else:
            df_aux.loc[indicesToKeep, 'atenciones'] = df_aux.loc[indicesToKeep, 'atenciones'] / sum(df_aux.loc[indicesToKeep, 'atenciones'])
df3 = df_aux.pivot(index=['year','CodigoRegion','grupo_etario'],columns= 'nrodia',values='atenciones').reset_index()

In [6]:
#analisis PCA
# Separating out the features
x_original = df3.loc[:, [k+1 for k in range(periodo)]].values

# Separating out the target
y = df3.loc[:,['CodigoRegion','grupo_etario']].values

#Standardizing the features
scaler = StandardScaler().fit(x_original)
x = scaler.transform(x_original)

pca = PCA(n_components=4)

principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2', 'PC3','PC4'])
finalDf = pd.concat([principalDf, df3[['CodigoRegion','grupo_etario']]], axis = 1)

In [7]:
#test for a good group in the classification
# here we test the different groupings to check if there 
# are others that make sense, since we test for binary classification 
#and we only have 5 classes it makes sense to consider the groupings
# (1), (2), (3), (4), (5), (1,2), (1,3), (1,4), (1,5), (2,3), (2,4), (2,5),(3,4),(3,5),(4,5)
#those can be presented in an array and they describe all the possibilities for binary classification

maxiter = 1000
resultados= np.zeros([maxiter,5,5])

for iter in range(maxiter):
    for group_indices in [[0], [1], [2], [3], [4], [0,1], [0,2], [0,3], [0,4], [1,2], [1,3], [1,4],[2,3],[2,4],[3,4]]:
        # Declare the group we are trying to separate
        targets = []
        #for ind in set([0,1,2,3,4]) - set(group_indices):
        for ind in group_indices:
            if len(targets) == 0:
                targets = (finalDf[["grupo_etario"]]==grupo_etario_list[ind]).values.ravel()
            else:
                targets = targets | (finalDf[["grupo_etario"]]==grupo_etario_list[ind]).values.ravel()
        
        X, y = finalDf[["PC1","PC2","PC3"]].values , targets 
        # Split the dataset into a training and a testing set(20 percent)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
        #print ("\nTesting Dataset Shape:", X_train.shape, y_train.shape)
        
        # Standarize the features
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # create the linear model SGDclassifier
        linear_clf = SGDClassifier(loss="hinge", alpha=0.000001, max_iter=1000)
        # Train the classifier using fit() function
        linear_clf.fit(X_train, y_train)
        # Evaluate the result 
        y_train_pred = linear_clf.predict(X_train)
        #precision_clasificador = metrics.f1_score(y_train, y_train_pred)
        precision_clasificador = metrics.balanced accuracy_score(y_train, y_train_pred)
        #presicion_clasificador = metrics.accuracy_score(y_train, y_train_pred)
        if len(group_indices)==1:
            resultados[iter, group_indices[0],group_indices[0]] = precision_clasificador
        else:
            resultados[iter, group_indices[0],group_indices[1]] = precision_clasificador
            resultados[iter, group_indices[1],group_indices[0]] = resultados[iter,group_indices[0],group_indices[1]]
#resultados.mean(0)
pd.DataFrame(resultados.mean(0),columns=grupo_etario_legend, index = grupo_etario_legend)



,Menores 1,De 1 a 4,De 5 a 14,De 15 a 64,De 65 y mas
Menores 1,0.140923,0.433054,0.470906,0.447733,0.524429
De 1 a 4,0.433054,0.280656,0.839044,0.380244,0.385250
De 5 a 14,0.470906,0.839044,0.527471,0.402762,0.450818
De 15 a 64,0.447733,0.380244,0.402762,0.235119,0.773839
De 65 y mas,0.524429,0.385250,0.450818,0.773839,0.573628
